# Loading data

In [ ]:
from numbers import Number
from collections import defaultdict

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import kosh
import math

%matplotlib notebook

# Initialization
database = './data/baseline_output.sqlite'
target_type = "csv_rec"
datastore = kosh.connect(database)
print("Kosh is ready!")

# Adding Data to Records

In [ ]:
def RMSE(x_true, x_pred):
    summation = 0
    for xt, xp in zip(x_true, x_pred):
        summation += (xt-xp)**2
    mean = summation/len(x_true)
    rmse = math.sqrt(mean)
    return rmse


def diff(x_true, x_pred):
    dif = []
    for xt, xp in zip(x_true, x_pred):
        dif.append(xt-xp)
    return dif

val = list(datastore.find(id_pool = '47bcda_0'))[0] #need to convert to list then extract
# Printing Attributes and Features
print('Attributes:')
print('\t',val.list_attributes())
print('\n')
print('Features Sets:')
print('\t',val.list_features())

# Validation Data
x_true = val['physics_cycle_series/x_pos'][:]
y_true = val['physics_cycle_series/y_pos'][:]
z_true = val['physics_cycle_series/z_pos'][:]
time = val['physics_cycle_series/time'][:]

# Adding RMSE and Diff data to each record
for dataset in datastore.find(): # Each record is now a dataset
    print(f"id: {dataset.id}")
    x_pred = dataset['physics_cycle_series/x_pos'][:]
    y_pred = dataset['physics_cycle_series/y_pos'][:]
    z_pred = dataset['physics_cycle_series/z_pos'][:]

    x_diff = diff(x_true, x_pred)
    x_rmse = RMSE(x_true, x_pred)
    y_diff = diff(y_true, y_pred)
    y_rmse = RMSE(y_true, y_pred)
    z_diff = diff(z_true, z_pred)
    z_rmse = RMSE(z_true, z_pred)

    print(f"\tx_rmse: {x_rmse}")
    print(f"\ty_rmse: {y_rmse}")
    print(f"\tz_rmse: {z_rmse}")

    setattr(dataset, 'x_rmse', x_rmse)
    dataset.add_curve(x_diff,'x_diff', 'physics_cycle_series')
    setattr(dataset, 'y_rmse', y_rmse)
    dataset.add_curve(y_diff,'y_diff', 'physics_cycle_series')
    setattr(dataset, 'z_rmse', z_rmse)
    dataset.add_curve(z_diff,'z_diff', 'physics_cycle_series')


# Parameter Table

In [ ]:
# Printing parameter table in Markdown
header = 1
for dataset in datastore.find(): # Each record is now a dataset

    if header == 1:
        print('| rec.id | x_pos_initial | y_pos_initial | z_pos_initial | x_vel_initial | y_vel_initial | z_vel_initial |')
        print('| --- | --- | --- | --- | --- | --- | --- |')
        header = 0
    print('|', dataset.id,
          '|', dataset.x_pos_initial,
          '|', dataset.y_pos_initial,
          '|', dataset.z_pos_initial,
          '|', dataset.x_vel_initial,
          '|', dataset.y_vel_initial,
          '|', dataset.z_vel_initial,
          '|'
          )

# Plotting QoIs and their Diff


In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))

for dataset in datastore.find(): # Each record is now a dataset
    ax[0, 0].plot(dataset['physics_cycle_series/time'][:], dataset['physics_cycle_series/x_pos'][:], label=dataset.id)
    ax[0, 1].plot(dataset['physics_cycle_series/time'][:], dataset['physics_cycle_series/y_pos'][:], label=dataset.id)
    ax[0, 2].plot(dataset['physics_cycle_series/time'][:], dataset['physics_cycle_series/z_pos'][:], label=dataset.id)
    
#     _ = vis.create_line_plot(fig=fig, ax=ax[1, 0], x="time", y="x_diff", title="{y_name}")
#     _ = vis.create_line_plot(fig=fig, ax=ax[1, 1], x="time", y="y_diff", title="{y_name}")
#     _ = vis.create_line_plot(fig=fig, ax=ax[1, 2], x="time", y="z_diff", title="{y_name}")

ax[0, 0].legend()
fig.savefig("./images/QoIs.png")

# RMSE Plots

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 10))

header = 1
for dataset in datastore.find(): # Each record is now a dataset
    ax[0].bar(x=dataset.id, height=dataset.x_rmse)
    ax[1].bar(x=dataset.id, height=dataset.y_rmse)
    ax[2].bar(x=dataset.id, height=dataset.z_rmse)

    if header == 1:
        print('| rec.id | x_rmse | y_rmse | z_rmse | Mean RMSE |')
        print('| --- | --- | --- | --- | --- |')
        header = 0
    print('|', dataset.id,
          '|', round(dataset.x_rmse, 2),
          '|', round(dataset.y_rmse, 2),
          '|', round(dataset.z_rmse, 2),
          '|', round((dataset.x_rmse+dataset.y_rmse+dataset.z_rmse)/3, 2),
          '|'
          )

ax[0].tick_params(rotation=45)
ax[1].tick_params(rotation=45)
ax[2].tick_params(rotation=45)

ax[0].set_title('x_rmse')
ax[1].set_title('y_rmse')
ax[2].set_title('z_rmse')

ax[0].set_ylabel('x_rmse')
ax[1].set_ylabel('y_rmse')
ax[2].set_ylabel('z_rmse')

fig.savefig("./images/RMSE.png")